# Step 1 - Deploy Sentiment Analysis Endpoint

Part of the stocks demo includes sentiment analysis on top of different financial feeds to elicit the market's sentiment towards the stock.  
To achieve that will use a [bert](https://arxiv.org/pdf/1810.04805.pdf) based language model pre-trained on our listed datasources.

### Configurations
Where do we want to save our models?

In [1]:
import os

In [12]:
# Configurations
project = 'stocks'
model_location = 'https://iguazio-sample-data.s3.amazonaws.com/models/model.pt'
saved_models_directory = os.path.join(os.path.abspath('../'), 'models')

# Create paths
os.makedirs(saved_models_directory, exist_ok=1)
model_filepath = os.path.join(saved_models_directory, os.path.basename(model_location))
print(f'Model file will be saved to {model_filepath}')

Model file will be saved to /User/demo-stocks/models/model.pt


We need to download the model to our designated directory

In [13]:
! wget -N -P {saved_models_directory} {model_location} 

--2020-09-27 13:28:51--  https://iguazio-sample-data.s3.amazonaws.com/models/model.pt
Resolving iguazio-sample-data.s3.amazonaws.com (iguazio-sample-data.s3.amazonaws.com)... 52.216.89.43
Connecting to iguazio-sample-data.s3.amazonaws.com (iguazio-sample-data.s3.amazonaws.com)|52.216.89.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433298364 (413M) [application/x-www-form-urlencoded]
Saving to: ‘/User/demo-stocks/models/model.pt’

model.pt            100%[===================>] 413.22M  9.34MB/s    in 44s     

2020-09-27 13:29:35 (9.38 MB/s) - ‘/User/demo-stocks/models/model.pt’ saved [433298364/433298364]



## Deploy sentiment function from hub

Next, we will import the [sentiment server](https://github.com/mlrun/functions/blob/master/sentiment_analysis_serving/bert_sentiment_analysis_serving.ipynb) from mlrun's function hub.

In [14]:
from mlrun import import_function, mount_v3io
server_fn = import_function(url='hub://sentiment_analysis_serving')

# Add the sentiment analysis model
server_fn.add_model('bert_classifier_v1', model_filepath)

# Add V3IO Mount
server_fn.apply(mount_v3io())

# Increase readiness timeout
server_fn.spec.readinessTimeoutSeconds = 500
server_fn.metadata.labels = {}

Finally, deploy it to the cluster.

In [15]:
server_fn.deploy(project='stocks')

> 2020-09-27 13:29:36,076 [info] deploy started
[nuclio] 2020-09-27 13:29:37,150 (info) Build complete
[nuclio] 2020-09-27 13:30:11,026 (info) Function deploy complete
[nuclio] 2020-09-27 13:30:11,032 done updating stocks-sentiment-analysis-serving, function address: 3.23.175.159:30328


'http://3.23.175.159:30328'

Please note the http endpoint above. We will use it later on.